# <div style='color:white;background: #005792;text-align: center;padding: 15px 0'>Recommandations - Transformation des données Title principals</div>

## Participants
* Samantha
* Rachelle
* Andrew


## <div style='background: #005792;text-align: center;padding: 15px 0'> <a style= 'color:white;' >Configuration des variables globales</a></div>

### Installation des librairies

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install plotly-express
# !pip install plotly

### Importation des librairies

In [4]:
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import re

### Chargement des fichiers

In [1]:
transformed_dir = '/home/dstrec/dstrec/010_data/001_transformed'

title_principals= 'title_principals.csv'
name_basics = 'name_basics.csv'

file_title_principals= f"{transformed_dir}/{title_principals}"
file_name_basics= f"{transformed_dir}/{name_basics}"

### Configuration des fonctions

In [2]:
def process_chunk(chunk):
    df_pivot = chunk.pivot_table(index='tconst', columns='category', values='primaryName', aggfunc='first')
    df_pivot.reset_index(inplace=True)
    df_pivot.columns = [col if col is not None else 'none' for col in df_pivot.columns]
    return df_pivot

## <div style='background: #005792;text-align: center;padding: 15px 0'> <a style= 'color:white;' >Préparation des données</a></div>

### Chargement du jeu de données

In [5]:
df_principals = pd.read_csv(file_title_principals)
df_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000009,1,nm0063086,actress,NaN,"[""Miss Geraldine Holbrook (Miss Jerry)""]"
1,tt0000009,2,nm0183823,actor,NaN,"[""Mr. Hamilton""]"
2,tt0000009,3,nm1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor..."
3,tt0000009,4,nm0085156,director,NaN,NaN
4,tt0000009,5,nm0085156,writer,NaN,NaN


In [6]:
df_principals.drop(columns=['ordering','job','characters'], inplace=True)

In [7]:
chunk_size = 100000
merged_chunks = []

for chunk in pd.read_csv(file_name_basics, chunksize=chunk_size):
    merged_chunk = pd.merge(df_principals, chunk, on='nconst', how='inner')
    merged_chunks.append(merged_chunk)

df_principals_cleaned = pd.concat(merged_chunks, ignore_index=True)

df_principals_cleaned.head()

,tconst,nconst,category,primaryName
0,tt0000009,nm0063086,actress,Blanche Bayliss
1,tt0000009,nm0085156,director,Alexander Black
2,tt0000009,nm0085156,writer,Alexander Black
3,tt0000009,nm0085156,producer,Alexander Black
4,tt0000009,nm0085156,cinematographer,Alexander Black


### EDA

In [8]:
df_principals_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841628 entries, 0 to 6841627
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   tconst       object
 1   nconst       object
 2   category     object
 3   primaryName  object
dtypes: object(4)
memory usage: 208.8+ MB


### Valeurs manquantes

In [9]:
df_principals_cleaned.isna().sum()

tconst         0
nconst         0
category       0
primaryName    0
dtype: int64

### Suppression de la colonne `nconst`

In [11]:
df_principals_cleaned.dropna(subset='primaryName')
df_principals_cleaned.drop(columns=['nconst'], inplace=True)

### Affichage du jeu de données

In [12]:
df_principals_cleaned.head()

,tconst,category,primaryName
0,tt0000009,actress,Blanche Bayliss
1,tt0000009,director,Alexander Black
2,tt0000009,writer,Alexander Black
3,tt0000009,producer,Alexander Black
4,tt0000009,cinematographer,Alexander Black


### Remodelage de la colonne `category`

In [13]:
df_principals_transformed = process_chunk(df_principals_cleaned)
df_principals_transformed.head()

,tconst,actor,actress,archive_footage,archive_sound,casting_director,cinematographer,composer,director,editor,producer,production_designer,self,writer
0,tt0000009,William Courtenay,Blanche Bayliss,NaN,NaN,NaN,Alexander Black,NaN,Alexander Black,NaN,Alexander Black,NaN,NaN,Alexander Black
1,tt0000147,NaN,NaN,NaN,NaN,NaN,Enoch J. Rector,NaN,Enoch J. Rector,NaN,William A. Brady,NaN,James J. Corbett,NaN
2,tt0000502,Antonio del Pozo,NaN,NaN,NaN,NaN,Ricardo de Baños,NaN,Ricardo de Baños,NaN,NaN,NaN,NaN,Ricardo de Baños
3,tt0000574,Godfrey Cass,Bella Cola,NaN,NaN,NaN,Millard Johnson,Eric Chapus,Charles Tait,NaN,W.A. Gibson,NaN,NaN,Charles Tait
4,tt0000675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Narciso Cuyàs,NaN,NaN,NaN,NaN,Miguel de Cervantes y Saavedra


### Valeurs manquantes

In [14]:
df_principals_transformed.isna().sum()

tconst                      0
actor                  159729
actress                194922
archive_footage        564082
archive_sound          572425
casting_director       513850
cinematographer        191577
composer               301291
director                46632
editor                 239323
producer               204624
production_designer    470261
self                   515752
writer                 131135
dtype: int64

### Suppression de colonnes

In [15]:
df_principals_transformed.drop(columns=['archive_footage', 'archive_sound', 'casting_director', 'cinematographer', 'production_designer','self', 'editor', 'writer'], inplace=True)

### Affichage du jeu de données

In [16]:
df_principals_transformed.head()

,tconst,actor,actress,composer,director,producer
0,tt0000009,William Courtenay,Blanche Bayliss,NaN,Alexander Black,Alexander Black
1,tt0000147,NaN,NaN,NaN,Enoch J. Rector,William A. Brady
2,tt0000502,Antonio del Pozo,NaN,NaN,Ricardo de Baños,NaN
3,tt0000574,Godfrey Cass,Bella Cola,Eric Chapus,Charles Tait,W.A. Gibson
4,tt0000675,NaN,NaN,NaN,Narciso Cuyàs,NaN


### EDA

In [17]:
df_principals_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572547 entries, 0 to 572546
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   tconst    572547 non-null  object
 1   actor     412818 non-null  object
 2   actress   377625 non-null  object
 3   composer  271256 non-null  object
 4   director  525915 non-null  object
 5   producer  367923 non-null  object
dtypes: object(6)
memory usage: 26.2+ MB


### Comparaison entre la colonne `director` et `producer`

In [18]:
filtered_df = df_principals_transformed.dropna(subset=['director', 'producer'])
comparison = filtered_df['director'] == filtered_df['producer']
percentage_identical = comparison.mean() * 100
percentage_identical

30.634548643720542

In [19]:
different_director_producer = filtered_df[~comparison]
different_director_producer.head()

,tconst,actor,actress,composer,director,producer
1,tt0000147,NaN,NaN,NaN,Enoch J. Rector,William A. Brady
3,tt0000574,Godfrey Cass,Bella Cola,Eric Chapus,Charles Tait,W.A. Gibson
5,tt0000679,L. Frank Baum,NaN,Nathaniel D. Mann,Francis Boggs,L. Frank Baum
22,tt0001028,NaN,NaN,NaN,Theo Frenkel,Cecil M. Hepworth
47,tt0001370,Eduardo Brazão,Amelia Vieira,NaN,Carlos Santos,Júlio Costa


In [20]:
df_principals_transformed.isna().sum()

tconst           0
actor       159729
actress     194922
composer    301291
director     46632
producer    204624
dtype: int64

### Insertion dans un fichier CSV

In [21]:
dest_dir = '/home/dstrec/dstrec/010_data/001_transformed'
title_principals_transformed_csv = 'title_principals_transformed.csv'

output_file_title_principals = f"{dest_dir}/{title_principals_transformed_csv}"

df_principals_transformed.to_csv(output_file_title_principals, index=False)